In [1]:
import numpy as np
import pandas as pd

In [2]:
train_sc01 = pd.read_csv('train_sc01.csv')

In [3]:
test_sc01 = pd.read_csv('test_sc01.csv')

In [4]:
train_dataType = train_sc01.dtypes
train_dataType.value_counts()

float64    1959
object        3
int64         1
dtype: int64

In [5]:
def clean_data(data):
    uniq_id = data['SampleID']
    label = data['GA']
    data.drop(columns=['SampleID','data','randPerson'])
    x = data.drop(columns=['SampleID','data','randPerson', 'GA'])
    return x, label

In [6]:
x_train, y_train = clean_data(train_sc01)
x_test, y_test = clean_data(test_sc01)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_X = scaler.fit_transform(x_train)
test_X = scaler.transform(x_test)

# Training ML Models

In [8]:
from sklearn.linear_model import LinearRegression

predictor = LinearRegression(n_jobs=-1)
predictor.fit(train_X, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [9]:
outcome = predictor.predict(test_X)
coefficients = predictor.coef_

In [10]:
from sklearn import metrics

In [11]:
def analysis(y_test, y_pred):
    print('MAE:',metrics.mean_absolute_error(y_test, y_pred))
    print('MSE:',metrics.mean_squared_error(y_test, y_pred))
    print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [12]:
analysis(y_test,outcome)

MAE: 5.63935050169577
MSE: 58.30399568680352
RMSE: 7.635705316917588


In [16]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 5000 decision trees
rf = RandomForestRegressor(n_estimators = 5000, random_state = 42)
# Train the model on training data
rf.fit(train_X, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=5000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [17]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_X)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 1.36


In [18]:
analysis(y_test,predictions)

MAE: 1.3582380952380777
MSE: 4.516062123673232
RMSE: 2.125102850140019


In [19]:
y_test

0     11.000000
1     11.000000
2     11.000000
3     15.000000
4     18.000000
5     16.000000
6     25.000000
7     27.000000
8     27.000000
9      2.571429
10     0.571429
11     0.571429
Name: GA, dtype: float64

In [20]:
predictions

array([11.93125714, 11.41345714, 11.63765714, 15.47177143, 16.72714286,
       16.80854286, 24.76205714, 26.52951429, 23.89991429,  1.27151429,
        0.11882857,  6.77371429])

In [27]:
submission = pd.DataFrame()
submission['SampleID'] = test_sc01['SampleID']
submission['GA'] = y_test
submission['predicted.GA'] = predictions
submission

,SampleID,GA,predicted.GA
0,I1,11.000000,11.931257
1,W1,11.000000,11.413457
2,B1,11.000000,11.637657
3,I2,15.000000,15.471771
4,W2,18.000000,16.727143
5,B2,16.000000,16.808543
6,I3,25.000000,24.762057
7,W3,27.000000,26.529514
8,B3,27.000000,23.899914
9,I4,2.571429,1.271514


In [28]:
submission.to_csv('GROUP-5--SC1--RandomForest.csv')